<a href="https://colab.research.google.com/github/SMotaee/Deep-Learning/blob/main/Diabetes%20disease%20detection%20using%20transfer%20learning%20method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import preprocess_input
from sklearn.model_selection import train_test_split

# Load the dataset
data = pd.read_csv('diabetes.csv')

# Split the dataset into training and testing sets
train, test = train_test_split(data, test_size=0.2, random_state=42)

# Extract the features and labels for the training set
train_features = train.drop(['Outcome'], axis=1).values
train_labels = train['Outcome'].values

# Extract the features and labels for the testing set
test_features = test.drop(['Outcome'], axis=1).values
test_labels = test['Outcome'].values

# Preprocess the input features
train_features = preprocess_input(train_features)
test_features = preprocess_input(test_features)


In [ ]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Load the Inception V3 model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add new layers for classification
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

# Create the transfer learning model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))

# Save the model
model.save('diabetes_detection_model.h5')


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score

# Load the trained model
model = load_model('diabetes_detection_model.h5')

# Load the dataset
data = pd.read_csv('diabetes.csv')

# Extract the features and labels
features = data.drop(['Outcome'], axis=1).values
labels = data['Outcome'].values

# Preprocess the input features
features = preprocess_input(features)

# Evaluate the model on the dataset
predictions = model.predict(features)
predictions = np.round(predictions).flatten()
accuracy = accuracy_score(labels, predictions)

print("Accuracy: {:.2f}%".format(accuracy * 100))